In [196]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [197]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [198]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [199]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [200]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [201]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [202]:
X['Date'] = X.index

In [203]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [204]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [205]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [206]:
len(y_sample), len(X_sample)

(1252, 1252)

In [207]:
torch.set_default_dtype(torch.float64)

In [225]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [209]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [210]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [211]:
# train_data = pd.DataFrame(train_data)
# train_data.iloc[0, 0] = torch.nan_to_num(train_data.iloc[0, 0], nan=1)

In [229]:
temp_tensor = train_data[0][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_1 = (temp_tensor, train_data[0][1])


temp_tensor = train_data[1][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_2 = (temp_tensor, train_data[1][1])

In [231]:
train_data[0] = temp_tuple_1
train_data[1] = temp_tuple_2

In [232]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32,)
test_dataloader= DataLoader(test_data, batch_size=32,)

In [233]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        print(X)
        print(torch.nan_to_num(X, nan=0))
        count+=1
    batch +=1


In [234]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [235]:
mlpmodel = MLP(8, 30, 1)

In [240]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.01)

In [241]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 0.018686001060709768
We're on batch: 2 + Loss: 0.009568517808925418
We're on batch: 3 + Loss: 0.007290692561760385
We're on batch: 4 + Loss: 0.010521609843332405
We're on batch: 5 + Loss: 0.009241292764047601
We're on batch: 6 + Loss: 0.007926023121389434
We're on batch: 7 + Loss: 0.00682580352536242
We're on batch: 8 + Loss: 0.006046687763443823
We're on batch: 9 + Loss: 0.005574371138554862
We're on batch: 10 + Loss: 0.00687051922783638
We're on batch: 11 + Loss: 0.007738457011300712
We're on batch: 12 + Loss: 0.0074718178066355044
We're on batch: 13 + Loss: 0.007012601974396125
We're on batch: 14 + Loss: 0.006701598840697632
We're on batch: 15 + Loss: 0.0067983740629937945
We're on batch: 16 + Loss: 0.0064557856129608855
We're on batch: 17 + Loss: 0.00610742095503191
We're on batch: 18 + Loss: 0.006179027061893583
We're on batch: 19 + Loss: 0.006745463582121388
We're on batch: 20 + Loss: 0.006848678099327897
We're on batch: 21 